<center>

# Web Intelligence

# Who will win the Tennis Australian Open 2020?

## 03 Building The Prediction Model

### Riccardo Spolaor (864877)

</center>

Modifico il dataframe in modo da poter ottenere un modello adatto alla predizione:
* Rinomino ***Winner*** in ***PlayerA*** e ***Loser*** in ***PlayerB***;
* Rinomino tutte le colonne con informazioni legate al vincitore e al perdente in modo da riflettere il legame ai nuovi nomi assegnati;
* Creo la colonna ***Winner*** che assume i valori:
    * 0 se ha vinto ***PlayerA***;
    * 1 se ha vinto ***PlayerB***;
* Assegno a ***Winner*** tutti valori 0;
* Duplico il dataframe e faccio lo swapping tra tutte le colonne con informazioni sul Giocatore A e quelle con informazioni sul Giocatore B;
* Assegno il valore 1 a ogni entry della colonna ***Winner*** del nuovo dataframe;
* Unisco i due dataframe.

In [319]:
import pandas as pd

In [320]:
dataframe = pd.read_csv('./datasets/dataframe05.csv', low_memory = False)

In [321]:
dataframe.columns.values

array(['csvID', 'ATP', 'Location', 'Tournament', 'Date', 'Series',
       'Court', 'Round', 'Best of', 'Winner', 'Loser', 'PlayedA',
       'PlayedB', 'WinRatioA', 'WinRatioB', 'WRank', 'WRankAdded',
       'LRank', 'LRankAdded', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4',
       'L4', 'W5', 'L5', 'Wsets', 'Lsets', 'FadigueTournGamesA',
       'FadigueTournGamesB', 'FadigueTournSetsA', 'FadigueTournSetsB',
       'WPts', 'WPtsAdded', 'LPts', 'LPtsAdded', 'MaxW', 'MaxWAdded',
       'MaxL', 'PlayedCourtA', 'PlayedCourtB', 'WonRatioCourtA',
       'WonRatioCourtB', 'MaxLAdded', 'AvgW', 'AvgWAdded', 'AvgL',
       'AvgLAdded', 'Carpet', 'Clay', 'Grass', 'Hard', 'Awarded',
       'Completed', 'Disqualified', 'Retired', 'Sched', 'Walkover',
       'Federer R.A', 'Federer R.B', 'Nadal R.A', 'Nadal R.B',
       'Djokovic N.A', 'Djokovic N.B', 'Murray A.A', 'Murray A.B',
       'Roddick A.A', 'Roddick A.B', 'Ferrer D.A', 'Ferrer D.B',
       'Berdych T.A', 'Berdych T.B', 'Hewitt L.A', 'Hewitt L.

In [322]:
def renameColumns():
    dataframe.rename(columns={'Winner':'PlayerA', 
                          'Loser':'PlayerB',
                          'WRank':'RankA',
                          'WRankAdded':'RankFilledA',
                          'LRank':'RankB',
                          'LRankAdded':'RankFilledB',
                          'W1':'1A',
                          'L1':'1B',
                          'W2':'2A',
                          'L2':'2B',
                          'W3':'3A',
                          'L3':'3B',
                          'W4':'4A',
                          'L4':'4B',
                          'W5':'5A',
                          'L5':'5B',
                          'Wsets':'setsA',
                          'Lsets':'setsB',
                          'WPts':'PtsA',
                          'LPts':'PtsB',
                          'WPtsAdded':'PtsFilledA',
                          'LPtsAdded':'PtsFilledB',
                          'MaxW':'MaxA',
                          'MaxWAdded':'MaxFilledA',
                          'MaxL':'MaxB',
                          'MaxLAdded':'MaxFilledB',
                          'AvgW':'AvgA',
                          'AvgWAdded':'AvgFilledA',
                          'AvgL':'AvgB',
                          'AvgLAdded':'AvgFilledB',
                         }, inplace = True)
    dataframe.insert(0, column ='Winner', value = 0)

In [323]:
renameColumns()
dataframe.columns.values

array(['Winner', 'csvID', 'ATP', 'Location', 'Tournament', 'Date',
       'Series', 'Court', 'Round', 'Best of', 'PlayerA', 'PlayerB',
       'PlayedA', 'PlayedB', 'WinRatioA', 'WinRatioB', 'RankA',
       'RankFilledA', 'RankB', 'RankFilledB', '1A', '1B', '2A', '2B',
       '3A', '3B', '4A', '4B', '5A', '5B', 'setsA', 'setsB',
       'FadigueTournGamesA', 'FadigueTournGamesB', 'FadigueTournSetsA',
       'FadigueTournSetsB', 'PtsA', 'PtsFilledA', 'PtsB', 'PtsFilledB',
       'MaxA', 'MaxFilledA', 'MaxB', 'PlayedCourtA', 'PlayedCourtB',
       'WonRatioCourtA', 'WonRatioCourtB', 'MaxFilledB', 'AvgA',
       'AvgFilledA', 'AvgB', 'AvgFilledB', 'Carpet', 'Clay', 'Grass',
       'Hard', 'Awarded', 'Completed', 'Disqualified', 'Retired', 'Sched',
       'Walkover', 'Federer R.A', 'Federer R.B', 'Nadal R.A', 'Nadal R.B',
       'Djokovic N.A', 'Djokovic N.B', 'Murray A.A', 'Murray A.B',
       'Roddick A.A', 'Roddick A.B', 'Ferrer D.A', 'Ferrer D.B',
       'Berdych T.A', 'Berdych T.B', 'He

In [324]:
def getInvertedDataFrame(colToInvert):
        
    players = dataframe['PlayerA'].value_counts().subtract(
        dataframe['PlayerB'].value_counts(), fill_value = 0).sort_values(ascending = False).index.tolist()[:25]
    
    colToInvert += players
    
    dataframeInverted = dataframe.copy(deep = True)

    dataframeInverted['Winner'] = 1
    
    for f in colToInvert:
        dataframeInverted[[f+'A', f+'B']] = dataframeInverted[[f+'B', f+'A']]
        
    dataframeInverted['OpponentsWRatioA>OpponentsWRatioB'] = dataframeInverted.apply(lambda x : int(x['OpponentsWRatioA'] > x['OpponentsWRatioB']), axis = 1)
    dataframeInverted['RankA>RankB'] = dataframeInverted.apply(lambda x : int(x['RankA'] > x['RankB']), axis = 1)
    dataframeInverted['AvgA>AvgB'] = dataframeInverted.apply(lambda x : int(x['AvgA'] > x['AvgB']), axis = 1)
    dataframeInverted['MaxA>MaxB'] = dataframeInverted.apply(lambda x : int(x['MaxA'] > x['MaxB']), axis = 1)
    dataframeInverted['WinRatioA>WinRatioB'] = dataframeInverted.apply(lambda x : int(x['WinRatioA'] > x['WinRatioB']), axis = 1)
    dataframeInverted['WonRatioCourtA>WonRatioCourtB'] = dataframeInverted.apply(lambda x : int(x['WonRatioCourtA'] > x['WonRatioCourtB']), axis = 1)

    dataframeInverted['FadigueTournGamesA>FadigueTournGamesB'] = dataframeInverted.apply(lambda x : int(x['FadigueTournGamesA'] > x['FadigueTournGamesB']), axis = 1)
    dataframeInverted['FadigueTournSetsA>FadigueTournSetsB'] = dataframeInverted.apply(lambda x : int(x['FadigueTournSetsA'] > x['FadigueTournSetsB']), axis = 1)
    
    return dataframeInverted

In [325]:
dataframeInverted = getInvertedDataFrame(['Player', 'Rank', 'RankFilled', 'Pts', 'PtsFilled', '1', '2', '3', '4', '5', 
                                          'sets', 'Max', 'Avg', 'Other', 'MaxFilled', 'AvgFilled',  'Played', 'WinRatio',
                                           'WonRatioCourt', 'PlayedCourt', 'OpponentsWRatio',
                                          'FadigueTournGames', 'FadigueTournSets', 'WalkoverLast', 'RetiredLast','5_1Mean',
                                          '5_2Mean', '5_3Mean', '5_4Mean', '5_5Mean', '5_setsMean'])




In [326]:
dataframeInverted.head(40)

Winner  csvID  ATP  Location  Tournament        Date  Series  Court  \
0        1      0  1.0         2           0  2001-01-01       0      1   
1        1      0  1.0         2           0  2001-01-01       0      1   
2        1      0  1.0         2           0  2001-01-01       0      1   
3        1      0  1.0         2           0  2001-01-01       0      1   
4        1      0  1.0         2           0  2001-01-01       0      1   
5        1      0  1.0         2           0  2001-01-01       0      1   
6        1      0  1.0         2           0  2001-01-01       0      1   
7        1      0  1.0         2           0  2001-01-01       0      1   
8        1      0  1.0         2           0  2001-01-01       0      1   
9        1      0  1.0         2           0  2001-01-01       0      1   
10       1      0  1.0         2           0  2001-01-01       0      1   
11       1      0  1.0         2           0  2001-01-01       0      1   
12       1      0  1.0         2           0  2001-01-01       0      1   
13       1      0  1.0         2           0  2001-01-01       0      1   
14       1      0  1.0         2           0  2001-01-01       0      1   
15       1      0  1.0         2           0  2001-01-01       0      1   
16       1      0  1.0         2           0  2001-01-01       0      1   
17       1      0  1.0         2           0  2001-01-01       0      1   
18       1      0  1.0         2           0  2001-01-01       0      1   
19       1      0  1.0         2           0  2001-01-01       0      1   
20       1      0  1.0         2           0  2001-01-01       0      1   
21       1      0  1.0         2           0  2001-01-01       0      1   
22       1      0  1.0         2           0  2001-01-01       0      1   
23       1      0  1.0         2           0  2001-01-01       0      1   
24       1      0  1.0         2           0  2001-01-01       0      1   
25       1      0  1.0         2           0  2001-01-01       0      1   
26       1      0  1.0         2           0  2001-01-01       0      1   
27       1      0  1.0         2           0  2001-01-01       0      1   
28       1      0  1.0         2           0  2001-01-01       0      1   
29       1      0  1.0         2           0  2001-01-01       0      1   
30       1      0  1.0         2           0  2001-01-01       0      1   
31       1      0  2.0        22           0  2001-01-01       0      1   
32       1      0  2.0        22           0  2001-01-01       0      1   
33       1      0  2.0        22           0  2001-01-01       0      1   
34       1      0  2.0        22           0  2001-01-01       0      1   
35       1      0  2.0        22           0  2001-01-01       0      1   
36       1      0  2.0        22           0  2001-01-01       0      1   
37       1      0  2.0        22           0  2001-01-01       0      1   
38       1      0  2.0        22           0  2001-01-01       0      1   
39       1      0  2.0        22           0  2001-01-01       0      1   

    Round  Best of  ... 5_3MeanA 5_4MeanA  5_5MeanA  5_setsMeanA  5_1MeanB  \
0       0      3.0  ...      0.0      0.0       0.0          0.0  0.000000   
1       0      3.0  ...      0.0      0.0       0.0          0.0  0.000000   
2       0      3.0  ...      0.0      0.0       0.0          0.0  0.000000   
3       0      3.0  ...      0.0      0.0       0.0          0.0  0.000000   
4       0      3.0  ...      0.0      0.0       0.0          0.0  0.000000   
5       0      3.0  ...      0.0      0.0       0.0          0.0  0.000000   
6       0      3.0  ...      0.0      0.0       0.0          0.0  0.000000   
7       0      3.0  ...      0.0      0.0       0.0          0.0  0.000000   
8       0      3.0  ...      0.0      0.0       0.0          0.0  0.000000   
9       0      3.0  ...      0.0      0.0       0.0          0.0  0.000000   
10      0      3.0  ...      0.0      0.0       0.0          0.0  0.00

In [327]:
with pd.option_context('display.max_columns',None):
    display(dataframeInverted[dataframeInverted['PlayerA'] == 'Nadal R.'].tail(2))
    display(dataframe[dataframe['PlayerB'] == 'Nadal R.'].tail(2))
    
#gamesWinRatioTourn pare avere problemi

Winner  csvID   ATP  Location  Tournament        Date  Series  Court  \
50962       1     18  39.0        49          25  2019-07-12       4      1   
51928       1     18  65.0        76          19  2019-11-02       2      0   

       Round  Best of   PlayerA        PlayerB  PlayedA  PlayedB  WinRatioA  \
50962      6      5.0  Nadal R.     Federer R.     1123     1339   0.826358   
51928      6      3.0  Nadal R.  Shapovalov D.     1139      138   0.827919   

       WinRatioB  RankA  RankFilledA  RankB  RankFilledB   1A   1B   2A   2B  \
50962   0.843167    2.0            0    3.0            0  6.0  7.0  6.0  1.0   
51928   0.550725    2.0            0   28.0            0  0.0  0.0  0.0  0.0   

        3A   3B   4A   4B   5A   5B  setsA  setsB  FadigueTournGamesA  \
50962  3.0  6.0  4.0  6.0  0.0  0.0    1.0    3.0               146.0   
51928  0.0  0.0  0.0  0.0  0.0  0.0    0.0    0.0                62.0   

       FadigueTournGamesB  FadigueTournSetsA  FadigueTournSetsB    PtsA  \
50962               153.0               16.0               17.0  7945.0   
51928                75.0                6.0                8.0  9225.0   

       PtsFilledA    PtsB  PtsFilledB  MaxA  MaxFilledA  MaxB  PlayedCourtA  \
50962           0  6620.0           0  1.77           0  2.30            90   
51928           0  1460.0           0  1.21           0  5.63           574   

       PlayedCourtB  WonRatioCourtA  WonRatioCourtB  MaxFilledB  AvgA  \
50962           199        0.788889        0.899497           0  1.71   
51928           106        0.775261        0.603774           0  1.18   

       AvgFilledA  AvgB  AvgFilledB  Carpet  Clay  Grass  Hard  Awarded  \
50962           0  2.18           0       0     0      1     0        0   
51928           0  4.94           0       0     0      0     1        0   

       Completed  Disqualified  Retired  Sched  Walkover  Federer R.A  \
50962          1             0        0      0         0            0   
51928          0             0        0      0         1            0   

       Federer R.B  Nadal R.A  Nadal R.B  Djokovic N.A  Djokovic N.B  \
50962            1          1          0             0             0   
51928            0          1          0             0             0   

       Murray A.A  Murray A.B  Roddick A.A  Roddick A.B  Ferrer D.A  \
50962           0           0            0            0           0   
51928           0           0            0            0           0   

       Ferrer D.B  Berdych T.A  Berdych T.B  Hewitt L.A  Hewitt L.B  \
50962           0            0            0           0           0   
51928           0            0            0           0           0   

       Del Potro J.M.A  Del Potro J.M.B  Tsonga J.W.A  Tsonga J.W.B  \
50962                0                0             0             0   
51928                0                0             0             0   

       Cilic M.A  Cilic M.B  Gasquet R.A  Gasquet R.B  Wawrinka S.A  \
50962          0          0            0            0             0   
51928          0          0            0            0             0   

       Wawrinka S.B  Nishikori K.A  Nishikori K.B  Monfils G.A  Monfils G.B  \
50962             0              0              0            0            0   
51928             0              0              0            0            0   

       Ferrero J.C.A  Ferrero J.C.B  Robredo T.A  Robredo T.B  Nalbandian D.A  \
50962              0              0            0            0               0   
51928              0              0            0            0               0   

       Nalbandian D.B  Raonic M.A  Raonic M.B  Agassi A.A  Agassi A.B  \
50962               0           0           0           0           0   
51928               0           0           0           0           0   

       Haas T.A  Haas T.B  Isner J.A  Isner J.B  Moya C.A  Moya C.B  \
50962         0         0          0          0         0         0   
51928         0        

Winner  csvID   ATP  Location  Tournament        Date  Series  Court  \
50962       0     18  39.0        49          25  2019-07-12       4      1   
51928       0     18  65.0        76          19  2019-11-02       2      0   

       Round  Best of        PlayerA   PlayerB  PlayedA  PlayedB  WinRatioA  \
50962      6      5.0     Federer R.  Nadal R.     1339     1123   0.843167   
51928      6      3.0  Shapovalov D.  Nadal R.      138     1139   0.550725   

       WinRatioB  RankA  RankFilledA  RankB  RankFilledB   1A   1B   2A   2B  \
50962   0.826358    3.0            0    2.0            0  7.0  6.0  1.0  6.0   
51928   0.827919   28.0            0    2.0            0  0.0  0.0  0.0  0.0   

        3A   3B   4A   4B   5A   5B  setsA  setsB  FadigueTournGamesA  \
50962  6.0  3.0  6.0  4.0  0.0  0.0    3.0    1.0               153.0   
51928  0.0  0.0  0.0  0.0  0.0  0.0    0.0    0.0                75.0   

       FadigueTournGamesB  FadigueTournSetsA  FadigueTournSetsB    PtsA  \
50962               146.0               17.0               16.0  6620.0   
51928                62.0                8.0                6.0  1460.0   

       PtsFilledA    PtsB  PtsFilledB  MaxA  MaxFilledA  MaxB  PlayedCourtA  \
50962           0  7945.0           0  2.30           0  1.77           199   
51928           0  9225.0           0  5.63           0  1.21           106   

       PlayedCourtB  WonRatioCourtA  WonRatioCourtB  MaxFilledB  AvgA  \
50962            90        0.899497        0.788889           0  2.18   
51928           574        0.603774        0.775261           0  4.94   

       AvgFilledA  AvgB  AvgFilledB  Carpet  Clay  Grass  Hard  Awarded  \
50962           0  1.71           0       0     0      1     0        0   
51928           0  1.18           0       0     0      0     1        0   

       Completed  Disqualified  Retired  Sched  Walkover  Federer R.A  \
50962          1             0        0      0         0            1   
51928          0             0        0      0         1            0   

       Federer R.B  Nadal R.A  Nadal R.B  Djokovic N.A  Djokovic N.B  \
50962            0          0          1             0             0   
51928            0          0          1             0             0   

       Murray A.A  Murray A.B  Roddick A.A  Roddick A.B  Ferrer D.A  \
50962           0           0            0            0           0   
51928           0           0            0            0           0   

       Ferrer D.B  Berdych T.A  Berdych T.B  Hewitt L.A  Hewitt L.B  \
50962           0            0            0           0           0   
51928           0            0            0           0           0   

       Del Potro J.M.A  Del Potro J.M.B  Tsonga J.W.A  Tsonga J.W.B  \
50962                0                0             0             0   
51928                0                0             0             0   

       Cilic M.A  Cilic M.B  Gasquet R.A  Gasquet R.B  Wawrinka S.A  \
50962          0          0            0            0             0   
51928          0          0            0            0             0   

       Wawrinka S.B  Nishikori K.A  Nishikori K.B  Monfils G.A  Monfils G.B  \
50962             0              0              0            0            0   
51928             0              0              0            0            0   

       Ferrero J.C.A  Ferrero J.C.B  Robredo T.A  Robredo T.B  Nalbandian D.A  \
50962              0              0            0            0               0   
51928              0              0            0            0               0   

       Nalbandian D.B  Raonic M.A  Raonic M.B  Agassi A.A  Agassi A.B  \
50962               0           0           0           0           0   
51928               0           0           0           0           0   

       Haas T.A  Haas T.B  Isner J.A  Isner J.B  Moya C.A  Moya C.B  \
50962         0         0          0          0         0         0   
51928         0        

In [328]:
dataframe = dataframe.append(dataframeInverted, sort = False, ignore_index = False)

In [329]:
def sortMatchesByDate ():
    dataframe['Index'] = dataframe.index
    dataframe.sort_values(['csvID', 'ATP','Index'],inplace = True)
    dataframe.drop('Index', axis=1, inplace = True)
    
sortMatchesByDate()

In [330]:
dataframe = dataframe.reset_index(drop = True)
dataframe.head(50)[['ATP','PlayerA', 'PlayerB', 'Round','Date']]

ATP         PlayerA         PlayerB  Round        Date
0   1.0      Clement A.     Gaudenzi A.      0  2001-01-01
1   1.0     Gaudenzi A.      Clement A.      0  2001-01-01
2   1.0    Goldstein P.        Jones A.      0  2001-01-01
3   1.0        Jones A.    Goldstein P.      0  2001-01-01
4   1.0         Haas T.        Smith L.      0  2001-01-01
5   1.0        Smith L.         Haas T.      0  2001-01-01
6   1.0       Henman T.     Rusedski G.      0  2001-01-01
7   1.0     Rusedski G.       Henman T.      0  2001-01-01
8   1.0       Hewitt L.      Arthurs W.      0  2001-01-01
9   1.0      Arthurs W.       Hewitt L.      0  2001-01-01
10  1.0    Johansson T.     Mantilla F.      0  2001-01-01
11  1.0     Mantilla F.    Johansson T.      0  2001-01-01
12  1.0     Ljubicic I.     Sargsian S.      0  2001-01-01
13  1.0     Sargsian S.     Ljubicic I.      0  2001-01-01
14  1.0      Malisse X.     Woodruff C.      0  2001-01-01
15  1.0     Woodruff C.      Malisse X.      0  2001-01-01
16  1.0       Martin A.      Puentes G.      0  2001-01-01
17  1.0      Puentes G.       Martin A.      0  2001-01-01
18  1.0        Massu N.      Sluiter R.      0  2001-01-01
19  1.0      Sluiter R.        Massu N.      0  2001-01-01
20  1.0        Pavel A.    Gimelstob J.      0  2001-01-01
21  1.0    Gimelstob J.        Pavel A.      0  2001-01-01
22  1.0         Phau B.   Woodbridge T.      0  2001-01-01
23  1.0   Woodbridge T.         Phau B.      0  2001-01-01
24  1.0       Rochus C.         Ilie A.      0  2001-01-01
25  1.0         Ilie A.       Rochus C.      0  2001-01-01
26  1.0  Stoltenberg J.     Grosjean S.      0  2001-01-01
27  1.0     Grosjean S.  Stoltenberg J.      0  2001-01-01
28  1.0      Tarango J.     Kohlmann M.      0  2001-01-01
29  1.0     Kohlmann M.      Tarango J.      0  2001-01-01
30  1.0        Vinck C.         Popp A.      0  2001-01-01
31  1.0         Popp A.        Vinck C.      0  2001-01-01
32  1.0         Haas T.      Malisse X.      1  2001-01-01
33  1.0      Malisse X.         Haas T.      1  2001-01-01
34  1.0       Henman T.      Tarango J.      1  2001-01-01
35  1.0      Tarango J.       Henman T.      1  2001-01-01
36  1.0       Hewitt L.         Phau B.      1  2001-01-01
37  1.0         Phau B.       Hewitt L.      1  2001-01-01
38  1.0    Johansson T.        Vinck C.      1  2001-01-01
39  1.0        Vinck C.    Johansson T.      1  2001-01-01
40  1.0     Ljubicic I.        Pavel A.      1  2001-01-01
41  1.0        Pavel A.     Ljubicic I.      1  2001-01-01
42  1.0       Martin A.       Rochus C.      1  2001-01-01
43  1.0       Rochus C.       Martin A.      1  2001-01-01
44  1.0        Massu N.      Clement A.      1  2001-01-01
45  1.0      Clement A.        Massu N.      1  2001-01-01
46  1.0  Stoltenberg J.    Goldstein P.      1  2001-01-01
47  1.0    Goldstein P.  Stoltenberg J.      1  2001-01-01
48  1.0         Haas T.       Hewitt L.      5  2001-01-01
49  1.0       Hewitt L.         Haas T.      5  2001-01-01

In [331]:
dataframe.drop(['Date', 'PlayerA', 'PlayerB', 'csvID', '1A', '1B', '2A', '2B', '3A', '3B', '4A', 
                '4B', '5A', '5B', 'setsA', 'setsB', 'Awarded', 'Completed',
       'Disqualified', 'Retired', 'Sched', 'Walkover', 'PlayedA', 'PlayedB', 'PlayedCourtA', 'PlayedCourtB',
               'FadigueTournGamesB', 'FadigueTournSetsA', 'FadigueTournGamesA', 'FadigueTournSetsB'], axis = 1, inplace = True)

In [332]:
dataframe.to_csv('./datasets/dataframe0.csv', index = False)